# 🙋1B 模型需要多少显存？



### 一. 1B的概念？
1B指的是有10亿（1 billion）参数量

如果以float32存储的话，那就是每个参数用4字节，则1B模型要$4*10^{9}$个Bytes

由于
- 1GB = 1024 MB
- 1MB = 1024 KB
- 1KB = 1024 Bytes

则 $4*10^{10}/(1024*1024)= 3814 GB$ 或者 3.73 GB显存



In [5]:
MB=4*(10**9)/(1024**2)
print("1B模型以float32存储大约需要显存: %.2f MB" % MB)
GB=MB/1024
print("1B模型以float32存储大约需要显存: %.2f GB" % GB)


1B模型以float32存储大约需要显存: 3814.70 MB
1B模型以float32存储大约需要显存: 3.73 GB


#### 其他精度

| 数据类型       | 单个参数占用字节 | 说明（应用场景）                     |
|----------------|------------------|--------------------------------------|
| FP32（单精度） | 4 字节           | 高精度场景（如模型训练），占用空间最大 |
| FP16（半精度） | 2 字节           | 主流推理 / 训练场景，平衡精度与空间   |
| BF16（脑半精度）| 2 字节           | 类似 FP16，部分芯片（如 GPU）优化支持 |
| INT8（8 位整数） | 1 字节           | 低精度推理（如边缘设备、轻量化部署）  |
| INT4（4 位整数） | 0.5 字节         | 超轻量化场景（极致压缩，精度有损失）  |

### 二. 训练1B模型的显存开销有哪些？

如果只使用随机梯度下降（SGD）的优化方法，那么显存的开销只有三部分：
- 模型参数 :1B参数
- 梯度：反向传播时，每个参数计算1个梯度，即1B参数
- 前向传播中间激活值，这和batchsize，seqlenth相关。

而如果用AdamW 优化器，还要新增加2B参数作为优化器状态。



#### 对比SGD和 AdamW 的更新公式：

- SGD: $ \theta_t = \theta_{t-1} - \eta \cdot g_t $


- AdamW: $  \theta_t = \theta_{t-1} - \eta \cdot \left( \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} + \lambda \theta_{t-1} \right) $
    - $\theta_t$ 是第 $t$ 步更新后的参数；
    - $\theta_{t-1}$ 是第 $t-1$ 步的参数；
    - $\eta$ 是学习率；
    - $g_t$ 是第 $t$ 步计算得到的梯度（基于单个样本或小批量样本计算的梯度）。
    - $m_t$  是当前的一阶矩估计（动量）， $  m_t = \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot g_t$
    - $v_t$  是当前的二阶矩估计（平方梯度的指数移动平均），$v_t = \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot g_t^2$


故而使用AdamW需要使用额外2倍空间




## 三. 待补充问题

中间激活值和batchsize，seqlength是怎么关联的，线性关系吗？